In [1]:
import tensorflow as tf
from tensorflow.keras import layers, regularizers, callbacks
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Data Preprocesing

In [2]:
def preprocess_data(x_train, y_train, config):
    x_train, x_valid, y_train, y_valid = train_test_split(x_resampled, y_resampled, test_size=0.2, random_state=42)
    smote = SMOTE(random_state=config['data'] ['resampling']['random_state'])
    x_resampled, y_resampled = smote.fit_resample(x_train, y_train)
    return x_train, x_valid, y_train, y_valid

# Model Building

In [3]:
def build_model(config):
    input_shape = [config['model']['input_shape']]
    model = tf.keras.Sequential()
    for layer in config['model']['layers']:
        if layer['type'] == 'Dense':
            model.add(layers.Dense(
                units=layer['units'], 
                activation=layer['activation'], 
                kernel_regularizer=layer.get('kernel_regularizer')
            ))
        elif layer['type'] == 'Dropout':
            model.add(layers.Dropout(rate=layer['rate']))
    model.summary()
    return model

# Compile Model

In [4]:
def compile_model(model, config):
    optimizer_config = config['training']['optimizer']
    optimizer = tf.keras.optimizers.Adam(learning_rate=optimizer_config['learning_rate'])
    model.compile(
        optimizer=optimizer,
        loss=config['training']['loss'],
        metrics=config['training']['metrics']
    )

# Callbacks

In [5]:
def setup_callbacks(config):
    lr_scheduler = callbacks.ReduceLROnPlateau(**config['training']['callbacks']['reduce_lr_on_plateau'])
    early_stopping = callbacks.EarlyStopping(**config['training']['callbacks']['early_stopping'])
    return [lr_scheduler, early_stopping]

# Training

In [6]:
def train_model(model, x_train, y_train, x_valid, y_valid, config, callbacks_list):
    history = model.fit(
        x_train, y_train,
        validation_data=(x_valid, y_valid),
        batch_size=config['training']['batch_size'],
        epochs=config['training']['epochs'],
        callbacks=callbacks_list
    )
    return history

# Evaluation 

In [7]:
def plot_training_history(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

# Running The Pipeline

In [9]:
x_train, x_valid, y_train, y_valid = preprocess_data(x_train, y_train, CONFIG)

model = build_model(CONFIG)

compile_model(model, CONFIG)

callbacks_list = setup_callbacks(CONFIG)

history = train_model(model, x_train, y_train, x_valid, y_valid, CONFIG, callbacks_list)

plot_training_history(history)

NameError: name 'x_train' is not defined